## 1. Typed Dictionary

In [1]:
from typing import TypedDict

class Person(TypedDict):
    name: str
    age: int

In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Optional, Literal

In [5]:
load_dotenv()

model= ChatGroq(model="gemma2-9b-it")

In [16]:
# Create a class

class Review (TypedDict):
    summary: str
    sentiment: str
    rating: int


structured_model = model.with_structured_output(Review)

In [18]:
structured_model.invoke(""""
Good mobile but facing heating issue 😕😕
""")

{'rating': 3,
 'sentiment': 'negative',
 'summary': 'Good mobile but facing heating issue'}

### Annotation, Literal, Optional:

In [ ]:
from typing import Annotated, Literal, Optional

class Review (TypedDict):
    key_themes: Annotated[list[str], "Write down all the key themes discussed in the review in a list"]
    summary: Annotated[str, "A brief summary of the review"]
    sentiment: Annotated[Literal["positive", "negative","mixed"], "Return sentiment of the review either negative, positive or neutral"]
    pros: Annotated[Optional[list[str]], "Write down all the pros inside a list"]
    cons: Annotated[Optional[list[str]], "Write down all the cons inside a list"]
    rating: int

structured_model = model.with_structured_output(Review)

structured_model.invoke(''' 
Very ok quality product. Very tasty and crispy. Good packing. Please keep it up.
'''

)

{'cons': [],
 'key_themes': ['product quality', 'taste', 'crispiness', 'packaging'],
 'pros': ['Very ok quality product', 'Very tasty and crispy'],
 'rating': 4,
 'sentiment': 'positive',
 'summary': "A positive review praising the product's quality, taste, crispiness, and packaging."}

### Pass Multiple reviews:

In [75]:
from typing import Annotated, Literal, Optional

class Review (TypedDict):
    key_themes: Annotated[list[str], "Write down all the key themes discussed in the review in a list"]
    summary: Annotated[str, "A brief summary of the review"]
    sentiment: Annotated[Literal["positive", "negative","mixed"], "Return sentiment of the review either negative, positive or neutral"]
    pros: Annotated[Optional[list[str]], "Write down all the pros inside a list"]
    cons: Annotated[Optional[list[str]], "Write down all the cons inside a list"]
    rating: int

Reviews = ["Completely waste , fully disappointed about quality , strongly non reffer , please don’t purchase anyone , please visit local offline stores and purchase , 1299 and 4 days weiting time all waste", "These pistachios are of top quality—crunchy, fresh, and packed with flavor. Perfect as a healthy snack or to add to desserts and dishes. The packaging keeps them fresh for longer, and the taste is absolutely delightful. A great value for the price! Highly recommend!"]

structured_model = model.with_structured_output(Review)

for r in Reviews:
    data1= structured_model.invoke(r)
    print(data1)

{'rating': 1, 'sentiment': 'negative', 'summary': 'The product is completely disappointing. The quality is terrible, and the wait time was excessive (4 days).  I would strongly advise against purchasing this product.  Instead,  consider visiting a local store for better options.'}
{'cons': [], 'key_themes': ['quality', 'taste', 'freshness', 'versatility', 'value'], 'pros': ['crunchy', 'fresh', 'packed with flavor', 'healthy snack', 'great for desserts and dishes', 'packaging keeps them fresh', 'delicious taste', 'great value'], 'rating': 5, 'sentiment': 'positive', 'summary': 'The pistachios are top quality, crunchy, fresh, and flavorful. The packaging keeps them fresh, and they are a great value for the price.'}


# 2. Pydantic

In [77]:
#pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [84]:
from pydantic import BaseModel

class Student(BaseModel):
    name: str = 'John'

new_student= {}

student1= Student(**new_student)

print(student1)

name='John'


In [90]:
student1.name

'John'

### Optional Fields

In [95]:
from pydantic import BaseModel
from typing import Optional

class Student(BaseModel):
    name: str = 'John'
    age: Optional[int]=None

new_student= {'age': '45'}

student1= Student(**new_student)

print(student1)    

name='John' age=45


## Validate Emails

In [100]:
#pip install pydantic[email]


   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   -------------------- ------------------- 1/2 [email-validator]
   ---------------------------------------- 2/2 [email-validator]

Note: you may need to restart the kernel to use updated packages.


In [106]:
from pydantic import BaseModel, EmailStr
from typing import Optional

class Student(BaseModel):
    name: str = 'John'
    age: Optional[int]=None
    email: EmailStr

new_student= {'name':'Cena', 'age': 45, 'email': "abc@gmail.com"}

student1= Student(**new_student)
student1

Student(name='Cena', age=45, email='abc@gmail.com')

## Field( Function)

In [113]:
from pydantic import BaseModel, EmailStr, Field
from typing import Optional

class Student(BaseModel):

    name: str = 'john'
    age: Optional[int] = None
    email: EmailStr
    cgpa: float = Field(gt=0, lt=10, default=5, description='A decimal value representing the cgpa of the student')


new_student = {'age':'32', 'email':'abc@gmail.com', 'cgpa': 6.3}

student1 = Student(**new_student)
student1

Student(name='john', age=32, email='abc@gmail.com', cgpa=6.3)

In [ ]:
# Convert this into python dictionary:

student_dict = dict(student1)
student_dict

{'name': 'john', 'age': 32, 'email': 'abc@gmail.com', 'cgpa': 6.3}

In [117]:
student_dict.get('age')

32

In [121]:
#Convert to JSON:
student1_json= student1.model_dump_json()
student1_json

'{"name":"john","age":32,"email":"abc@gmail.com","cgpa":6.3}'

# Pydantic + with_structured_output

In [122]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

In [127]:
class Review(BaseModel):
    key_themes : list[str] = Field(description= "Write down all the key themes discussed in the review in a list")
    summary: str = Field(description= "A brief summary of the review")
    sentiment: Literal['Positive', 'Negative', "Mixed"] = Field(description= "Return sentiment of the review either negative, positive or Mixed")
    pros: Optional[list[str]] = Field(default=None, description="Write down all the pros inside a list")
    cons: Optional[list[str]] = Field(default=None, description="Write down all the cons inside a list")
    name: Optional[str] = Field(default=None, description="Write the name of the reviewer")

#default=None required for Optional value    


structured_model = model.with_structured_output(Review)

In [129]:
result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Randy Orton
""")

print(result)

key_themes=['Performance and Camera', 'Battery life and charging', 'Size and weight', 'Software and Price'] summary='The Samsung Galaxy S24 Ultra is a powerful phone with an incredible camera, but its size, weight, and price may be drawbacks for some users.' sentiment='Mixed' pros=['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'] cons=['Weight and size make it uncomfortable for one-handed use', "Samsung's One UI still comes with bloatware", 'The $1,300 price tag is expensive'] name='Randy Orton'


In [131]:
type(result)

__main__.Review

# JSON Schema

In [132]:
{
    "title": "student",
    "description": "schema about students",
    "type": "object",
    "properties":{
        "name":"string",
        "age":"integer"
    },
    "required":["name"] #Name of the student must be present
}

{'title': 'student',
 'description': 'schema about students',
 'type': 'object',
 'properties': {'name': 'string', 'age': 'integer'},
 'required': ['name']}

In [133]:
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


In [134]:
structured_model = model.with_structured_output(json_schema)

In [135]:
result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Randy Orton
""")

print(result)

{'cons': ['The weight and size make it a bit uncomfortable for one-handed use.', 'Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides?', 'The $1,300 price tag is also a hard pill to swallow.'], 'key_themes': ['Performance', 'Camera', 'Battery Life', 'S-Pen', 'Bloatware', 'Price'], 'name': 'Randy Orton', 'pros': ['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'], 'sentiment': 'pos', 'summary': 'The Samsung Galaxy S24 Ultra is a powerful and feature-rich phone with a stunning camera and long battery life, but its size, bloatware, and high price are drawbacks.'}


# Convert Pydantic to JSON Schema

In [136]:
class Review(BaseModel):
    key_themes : list[str] = Field(description= "Write down all the key themes discussed in the review in a list")
    summary: str = Field(description= "A brief summary of the review")
    sentiment: Literal['Positive', 'Negative', "Mixed"] = Field(description= "Return sentiment of the review either negative, positive or Mixed")
    pros: Optional[list[str]] = Field(default=None, description="Write down all the pros inside a list")
    cons: Optional[list[str]] = Field(default=None, description="Write down all the cons inside a list")
    name: Optional[str] = Field(default=None, description="Write the name of the reviewer")


In [139]:
Review.model_json_schema()

{'properties': {'key_themes': {'description': 'Write down all the key themes discussed in the review in a list',
   'items': {'type': 'string'},
   'title': 'Key Themes',
   'type': 'array'},
  'summary': {'description': 'A brief summary of the review',
   'title': 'Summary',
   'type': 'string'},
  'sentiment': {'description': 'Return sentiment of the review either negative, positive or Mixed',
   'enum': ['Positive', 'Negative', 'Mixed'],
   'title': 'Sentiment',
   'type': 'string'},
  'pros': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'},
    {'type': 'null'}],
   'default': None,
   'description': 'Write down all the pros inside a list',
   'title': 'Pros'},
  'cons': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'},
    {'type': 'null'}],
   'default': None,
   'description': 'Write down all the cons inside a list',
   'title': 'Cons'},
  'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'description': 'Write the name of the rev

In [140]:
type(Review.model_json_schema())

dict